In [10]:
import torch

# checkpoints_dir = "checkpoints"

netArc_checkpoint = torch.load('./checkpoints/Model.pt', map_location="cpu")

In [12]:
netArc_checkpoint['optimizer']

KeyError: 'optimizer'

In [9]:
netArc_checkpoint['optimizer'] = None

torch.save(netArc_checkpoint, "checkpoints/Arcface_compatible.tar")

In [3]:
import torch

torch.__version__

'1.13.1+cpu'

In [1]:
import torch



In [19]:
conv_layer.weight.flatten(1).shape

torch.Size([64, 288])

In [17]:
(conv_layer.weight * torch.ones((64, 32, 3, 3))).shape

torch.Size([64, 32, 3, 3])

In [58]:
input_example = torch.rand(16, 32, 64, 64)

In [60]:
# conv_layer = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
# original_output = conv_layer(input_example)

# conv1_layer = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
# conv2_layer = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)

# conv1_layer.weight.data = conv_layer.weight.data[:32, ...]
# conv2_layer.weight.data = conv_layer.weight.data[32:, ...]



In [46]:
conv1_layer = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
original_weights1 = conv1_layer.weight

conv2_layer = torch.nn.Conv2d(in_channels=64, out_channels=16, kernel_size=5)
original_weights2 = conv2_layer.weight

new_size = 32
alpha_weights1 = torch.nn.Parameter(torch.ones(64, new_size, 1, 1))
new_filters1 = torch.einsum('ijkl,imkl->mjkl', (original_weights1, alpha_weights1))
new_filters1.shape

torch.Size([32, 32, 3, 3])

In [61]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def prune_interacitive(working_type, **kwargs):
    # Your function that does something based on working_type and other kwargs
    print(f"Working Type: {working_type}, Other kwargs: {kwargs}")

# Widgets definition
working_type_widget = widgets.Dropdown(options=['default', 'integral'], description="Working Type:")
# Assuming other widgets are defined in these dictionaries:
dict_for_interact = {'widget_a': widgets.IntSlider(description='A:')}
dict_for_interact_inn = {'widget_b': widgets.FloatSlider(description='B:')}

# Interactive widget without direct display
interac = widgets.interactive(prune_interacitive, working_type=working_type_widget, **dict_for_interact, **dict_for_interact_inn)

# Initial display based on current "working_type_widget" value
def update_widgets(change):
    if change['new'] == 'integral':
        for widget in dict_for_interact.values():
            widget.layout.display = 'none'  # Hide
        for widget in dict_for_interact_inn.values():
            widget.layout.display = None  # Show by resetting to default
    else:
        for widget in dict_for_interact.values():
            widget.layout.display = None  # Show
        for widget in dict_for_interact_inn.values():
            widget.layout.display = 'none'  # Hide

# Set initial visibility
update_widgets({'new': working_type_widget.value})

# Observe changes in "working_type_widget" to update other widgets visibility
working_type_widget.observe(update_widgets, names='value')

# Display UI elements
display(working_type_widget)  # Display combo-box always
# Since we're manually controlling visibility, we need to display all widgets explicitly
for widget in dict_for_interact.values():
    display(widget)
for widget in dict_for_interact_inn.values():
    display(widget)
display(interac.children[-1])  # Display the button



Dropdown(description='Working Type:', options=('default', 'integral'), value='default')

IntSlider(value=0, description='A:')

FloatSlider(value=0.0, description='B:', layout=Layout(display='none'))

Output()

In [73]:
import ipywidgets as widgets
from IPython.display import display, clear_output

def prune_interactive(working_type, **kwargs):
    # Function logic that does something based on the working_type and other kwargs
    print(f"Working Type: {working_type}, Other kwargs: {kwargs}")
    clear_output()
    print(f"Working Type: {working_type}, Other kwargs: {kwargs}")

# Define a common wide layout for widgets
wide_layout = widgets.Layout(width='100%')  # You can adjust the width as per your requirement

# Define interactive widgets with the wide layout
working_type_widget = widgets.Dropdown(options=['default', 'integral'], description="Working Type:", layout=wide_layout)
dict_for_interact = {'widget_a': widgets.IntSlider(description='A:', layout=wide_layout)}
dict_for_interact_inn = {'widget_b': widgets.FloatSlider(description='B:', layout=wide_layout)}

# Create a button for manual trigger with the wide layout
execute_button = widgets.Button(description="Execute", layout=wide_layout)

# Creating an output widget, which does not typically need a layout adjustment
output = widgets.Output()

vbox = widgets.VBox([working_type_widget], layout=widgets.Layout(width='100%'))  # Ensuring the VBox itself is wide

def update_widgets_ui(*args):
    new_children = [working_type_widget]  # Always include the dropdown
    if working_type_widget.value == 'integral':
        new_children += list(dict_for_interact_inn.values())
    else:
        new_children += list(dict_for_interact.values())
    new_children.append(execute_button)  # Add the execute button to the new children
    new_children.append(output)  # Ensure the output widget is always last
    vbox.children = new_children

def on_execute_button_clicked(b):
    with output:
        clear_output(wait=True)
        # Extract non-'Output' widget values as kwargs
        kwargs = {}
        for widget in vbox.children:
            if hasattr(widget, 'value') and not isinstance(widget, (widgets.Output, widgets.Button)):
                kwargs[widget.description.strip(':')] = widget.value
        # Call the function with current widget values
        prune_interactive(working_type=working_type_widget.value, **kwargs)

execute_button.on_click(on_execute_button_clicked)
update_widgets_ui()  # Initial UI update
working_type_widget.observe(update_widgets_ui, 'value')  # Observe changes in the dropdown

display(vbox)


In [56]:
alpha_weights2 = torch.nn.Parameter(torch.ones(new_size, 16, 1, 1))

conv2_layer = torch.nn.Conv2d(in_channels=64, out_channels=16, kernel_size=5)
print(conv2_layer.weight.shape)

torch.einsum('ijkl,jmkl->mikl', (original_weights2, alpha_weights2)).shape

torch.Size([16, 64, 5, 5])


RuntimeError: einsum(): subscript i has size 32 for operand 1 which does not broadcast with previously seen size 16